# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
import hvplot.pandas
import pandas as pd
import requests
import folium

from api_keys import geoapify_key

In [2]:
file_path = "city_weather_data.csv"

city_data_df = pd.read_csv(file_path)

display(city_data_df.head())


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,siklos,45.8550,18.2975,280.49,75,0,2.06,HU,1708571586
1,katsuura,35.1333,140.3000,279.16,89,100,10.89,JP,1708571437
2,al khawr,25.7667,51.3167,290.26,62,0,4.72,QA,1708571586
3,albany,42.6001,-73.9662,273.38,71,29,0.89,US,1708571446
4,waitangi,-43.9535,-176.5597,289.16,79,46,3.13,NZ,1708571330


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
m = folium.Map(location=[0, 0], zoom_start=2)

for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 5,  # Adjust the divisor to control the size of the markers
        popup=row['City'],
        color='red',
        fill=True,
        fill_color='pink'
    ).add_to(m)

folium.TileLayer('Stamen Terrain').add_to(m)

m

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
filtered_cities_df = city_data_df[((city_data_df['Max Temp'] >= 294.15) & 
                                   (city_data_df['Max Temp'] <= 300.15)) & 
                                  (city_data_df['Wind Speed'] < 4.5) & 
                                  (city_data_df['Cloudiness'] == 0)]

filtered_cities_df = filtered_cities_df.dropna()

display(filtered_cities_df.head())


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,hawaiian paradise park,19.5933,-154.9731,298.08,70,0,4.12,US,1708571607
73,jeddah,21.5169,39.2192,295.07,68,0,2.57,SA,1708571614
97,trujillo,-8.1160,-79.0300,298.13,78,0,2.57,PE,1708571621
108,stephenville,32.2207,-98.2023,294.19,64,0,4.12,US,1708571625
213,ciudad lazaro cardenas,17.9583,-102.2000,297.15,76,0,1.29,MX,1708571657


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

hotel_df['Hotel Name'] = ""

hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
49,hawaiian paradise park,US,19.5933,-154.9731,70,
73,jeddah,SA,21.5169,39.2192,68,
97,trujillo,PE,-8.1160,-79.0300,78,
108,stephenville,US,32.2207,-98.2023,64,
213,ciudad lazaro cardenas,MX,17.9583,-102.2000,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
radius = 10000000 
params = {
    "type": "hotel", 
    "radius": radius,
}

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["filter"] = f"point:lat:{lat},lng:{lng}"
    params["bias"] = f"point:lat:{lat},lng:{lng}"
    
    base_url = "https://api.geoapify.com/v2/place-details"
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df.head()


Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
jeddah - nearest hotel: No hotel found
trujillo - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: No hotel found
san jose del cabo - nearest hotel: No hotel found
eldorado - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
musay'id - nearest hotel: No hotel found
chokwe - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: No hotel found
altania - nearest hotel: No hotel found
road town - nearest hotel: No hotel found
sainte-suzanne - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
49,hawaiian paradise park,US,19.5933,-154.9731,70,No hotel found
73,jeddah,SA,21.5169,39.2192,68,No hotel found
97,trujillo,PE,-8.1160,-79.0300,78,No hotel found
108,stephenville,US,32.2207,-98.2023,64,No hotel found
213,ciudad lazaro cardenas,MX,17.9583,-102.2000,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
map_center = [hotel_df.iloc[0]['Lat'], hotel_df.iloc[0]['Lng']]
map = folium.Map(location=map_center, zoom_start=10)

for index, row in hotel_df.iterrows():
    folium.Marker(location=[row['Lat'], row['Lng']], popup=row['City']).add_to(map)

map
